<div class="alert alert-block" style="border: 1px solid #455A64;background-color:#ECEFF1;">
본 자료 및 영상 컨텐츠는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 컨텐츠 및 컨텐츠 일부 문구등을 외부에 공개, 게시하는 것을 금지합니다. 특히 자료에 대해서는 저작권법을 엄격하게 적용하겠습니다.
</div>

### 0. Get data

### 1. train/test 데이터 임포트

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import string
import warnings
import missingno
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('bikesharing/train.csv')
df_test = pd.read_csv('bikesharing/test.csv')
df_all = pd.concat((df_train, df_test)).reset_index(drop=True)

In [ ]:
def split_df(df):
    return df[:10885], df[10886:]

### RMSLE 기반 예측을 위한 log 필드 추가
> RMSLE 가 log 로 계산되므로, 예측값 또한 log 값으로 계산되도록 하는 편이 보다 RMSLE 성능에 도움을 줌

In [ ]:
df_all['casual_log'] = np.log(df_all['casual'] + 1)
df_all['registered_log'] = np.log(df_all['registered'] + 1)
df_all['count_log'] = np.log(df_all['count'] + 1)

### 시간 필드 추가

In [ ]:
dt = pd.DatetimeIndex(df_all['datetime'])
df_all.set_index(dt, inplace=True)

df_all['date'] = dt.date
df_all['day'] = dt.day
df_all['month'] = dt.month
df_all['year'] = dt.year
df_all['hour'] = dt.hour
df_all['dow'] = dt.dayofweek

# 202402 업데이트: 최신 라이브러리에서는 weekofyear 가 지원되지 않음
# 기존 코드: df_all['woy'] = dt.weekofyear
df_all['woy'] = dt.isocalendar().week

### peak 타임 필드 추가

In [ ]:
def func(df_data):
    if df_data['workingday'] == 1:
        if (df_data['hour'] == 8) or (df_data['hour'] == 17) or (df_data['hour'] == 18):
            return 4
        elif (df_data['hour'] == 7) or (df_data['hour'] == 16) or (df_data['hour'] == 19): 
            return 3           
    else:
        if (df_data['hour'] >= 12 and df_data['hour'] <= 16):
            return 2
        elif (df_data['hour'] >= 10 and df_data['hour'] <= 19):
            return 1
    return 0

# 0 or ‘index’: 각 컬럼에 함수 적용, 1 or ‘columns’: 각 행에 함수 적용
df_all['peak'] = df_all.apply(func, axis=1)

In [ ]:
def func(df_data):
    # 2021.10.22 업데이트
    # 영상에서는 24 일부터 31 일까지를 적용하지만, 테스트 결과 대부분 확실히 쉬는 24일과 31일만 적용했을 때,
    # 보다 결과가 좋았기 때문에, 24일과 31일만 적용하였습니다.
    if (df_data['month'] == 12) and (df_data['day'] == 24 or df_data['day'] == 31):
            return 1
    return df_data['holiday']

df_all['holiday'] = df_all.apply(func, axis=1)

In [ ]:
def func(df_data):
    # 2021.10.22 업데이트
    # 영상에서는 24 일부터 31 일까지를 적용하지만, 테스트 결과 대부분 확실히 쉬는 24일과 31일만 적용했을 때,
    # 보다 결과가 좋았기 때문에, 24일과 31일만 적용하였습니다.
    if (df_data['month'] == 12) and (df_data['day'] == 24 or df_data['day'] == 31):
            return 0
    return df_data['workingday']

df_all['workingday'] = df_all.apply(func, axis=1)

### 온도, 풍속, 습도, 날씨 기반 fit & humid 필드 추가

In [ ]:
def func(df_data):
    if (df_data['weather'] <= 2 and df_data['windspeed'] <= 20):
        if (df_data['temp'] > 15 and df_data['temp'] <= 35):
            return 1
    return 0

df_all['fit'] = df_all.apply(func, axis=1)

In [ ]:
def func(df_data):
    if df_data['humidity'] >= 70:
            return 1
    return 0

df_all['humid'] = df_all.apply(func, axis=1)

### Metric

$$ RMSLE = \sqrt{\dfrac{\sum_{i=0}^N (log(y_i + 1) - log(\hat{y_i} + 1))^2 }{N}} $$ 

In [ ]:
from sklearn.metrics import make_scorer

# 202402 업데이트: 최신 라이브러리에서는 기존 코드에서 불필요한 긴 경고메세지가 나타날 수 있습니다
# 최신 라이브러리에서는 y_pred 와 y_actual 에 0 이하의 값이 들어가면, 에러가 날 수 있으므로,
# 해당 값이 0이상이 되도록 코드를 수정하였습니다.
# 기존 코드
'''
def get_rmsle(y_actual, y_pred):
    diff = np.log(y_pred + 1) - np.log(y_actual + 1)
    mean_error = np.square(diff).mean()
    return np.sqrt(mean_error)
'''

# 신규 코드
def get_rmsle(y_actual, y_pred):
    # 음수 값을 방지하기 위해, 최소한 0 이상의 값을 갖도록 함
    y_pred = np.maximum(0, y_pred)
    # 로그를 취하기 전에, 값이 0 이상임을 보장
    log_pred = np.log(np.maximum(0, y_pred) + 1)
    log_actual = np.log(np.maximum(0, y_actual) + 1)
    diff = log_pred - log_actual
    mean_error = np.square(diff).mean()
    return np.sqrt(mean_error)

### Model Evaluation
- 참고: np.log() 와 np.exp 는 역함수 
- 수학적인 부분보다, 다음과 같이 코드로 바로 이해하기로 함

In [ ]:
def predict_bikecount(model, select_columns):
    df_train, df_test = split_df(df_all)

    X_train = df_train[select_columns]
    y_train_cas = df_train['casual_log']
    y_train_reg = df_train['registered_log']
    X_test = df_test[select_columns]
    
    casual_model = model.fit(X_train, y_train_cas)
    y_pred_cas = casual_model.predict(X_test)
    y_pred_cas = np.exp(y_pred_cas) - 1

    registered_model = model.fit(X_train, y_train_reg)
    y_pred_reg = registered_model.predict(X_test)
    y_pred_reg = np.exp(y_pred_reg) - 1

    return y_pred_cas + y_pred_reg

### Model Evaluation Test: LinearRegression

In [ ]:
df_train, df_test = split_df(df_all)
ml_columns = [
    'season', 'holiday', 'workingday', 'weather', 'temp',
    'atemp', 'humidity', 'windspeed', 'day', 'month',
    'year', 'hour', 'dow', 'woy', 'peak', 'fit', 'humid'
]
X_train = df_train[ml_columns].copy()
y_train = df_train['count']
rmsle_scorer = make_scorer(get_rmsle, greater_is_better=False)

In [ ]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
ml_pred = predict_bikecount(lr_model, ml_columns)
df_test['count'] = ml_pred
final_df = df_test[['datetime', 'count']].copy()
final_df.to_csv('submissions_linear.csv', header=True, index=False)
!kaggle competitions submit -c bike-sharing-demand -f submissions_linear.csv -m "Message"

### Model Evaluation Test: Lasso Regression

In [25]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

hyperparams = {'max_iter': [1000, 1500, 2000, 2500, 3000], 
               'alpha': 1/np.array([0.1, 1, 2, 3, 4, 10, 30,100,200,300,400,800,900,1000])
}

lasso_grid=GridSearchCV(estimator = Lasso(), param_grid = hyperparams, 
                verbose=True, scoring=rmsle_scorer, cv=5, n_jobs=-1)

lasso_grid.fit(X_train, y_train)
print(lasso_grid.best_params_)

Fitting 5 folds for each of 70 candidates, totalling 350 fits
{'alpha': 10.0, 'max_iter': 1000}


In [26]:
lasso_model = lasso_grid.best_estimator_
ml_pred = predict_bikecount(lasso_model, ml_columns)
df_test['count'] = ml_pred
final_df = df_test[['datetime', 'count']].copy()
final_df.to_csv('submissions_lasso.csv', header=True, index=False)
!kaggle competitions submit -c bike-sharing-demand -f submissions_lasso.csv -m "Message"

### Model Evaluation Test: Ridge Regression

In [27]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

hyperparams = {'max_iter': [1000, 1500, 2000, 2500, 3000], 
               'alpha':[0.1, 1, 2, 3, 4, 10, 30,100,200,300,400,800,900,1000]
}

ridge_grid=GridSearchCV(estimator = Ridge(), param_grid = hyperparams, 
                verbose=True, scoring=rmsle_scorer, cv=5, n_jobs=-1)

ridge_grid.fit(X_train, y_train)
print(ridge_grid.best_params_)

Fitting 5 folds for each of 70 candidates, totalling 350 fits
{'alpha': 1000, 'max_iter': 1000}


In [28]:
ridge_model = ridge_grid.best_estimator_
ml_pred = predict_bikecount(ridge_model, ml_columns)
df_test['count'] = ml_pred
final_df = df_test[['datetime', 'count']].copy()
final_df.to_csv('submissions_ridge.csv', header=True, index=False)
!kaggle competitions submit -c bike-sharing-demand -f submissions_ridge.csv -m "Message"

### Model Evaluation Test: Random Forest Regressor

In [32]:
from sklearn.model_selection import GridSearchCV

# 202402 업데이트: 실행시간이 매우 많이 걸리므로, n_estimators 를 테스트용으로 확연히 줄였습니다. 
# 보다 좋은 성능값을 위해 개인 PC 환경에 맞추어, 해당 값을 늘려보시면 좋을 것 같습니다.
# 기존 코드: n_estimators = [800, 1000, 1200]
n_estimators = [10, 20, 30] # 신규 코드

max_depth = [10, 12, 15]
min_samples_split = [4, 5, 6]
min_samples_leaf = [4, 5, 6]

hyperparams = {'n_estimators': n_estimators, 'max_depth': max_depth, 
               'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}

rf_grid = GridSearchCV(estimator = RandomForestRegressor(), param_grid = hyperparams, 
                verbose=True, scoring=rmsle_scorer, cv=5, n_jobs=-1)

rf_grid.fit(X_train, y_train)
print(rf_grid.best_params_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
{'max_depth': 15, 'min_samples_leaf': 4, 'min_samples_split': 4, 'n_estimators': 30}


In [33]:
rf_model = rf_grid.best_estimator_
ml_pred = predict_bikecount(rf_model, ml_columns)
df_test['count'] = ml_pred
final_df = df_test[['datetime', 'count']].copy()
final_df.to_csv('submissions_rf.csv', header=True, index=False)
!kaggle competitions submit -c bike-sharing-demand -f submissions_rf.csv -m "Message"

### Model Evaluation Test: XGBoost Regressor

In [35]:
from xgboost import XGBRegressor # 회귀트리 모델
from sklearn.model_selection import GridSearchCV

# 202402 업데이트: 실행시간이 매우 많이 걸리므로, n_estimators 를 테스트용으로 확연히 줄였습니다. 
# 보다 좋은 성능값을 위해 개인 PC 환경에 맞추어, 해당 값을 늘려보시면 좋을 것 같습니다.
# 기존 코드: 'n_estimators': [250, 500]
hyperparams = {'nthread':[4],
              'learning_rate': [0.05, 0.1, 0.15], 
              'max_depth': [4, 5],
              'min_child_weight': [3, 4, 5],
              'subsample': [0.7, 0.8],
              'colsample_bytree': [0.6, 0.7],
              'n_estimators': [10, 20]} # 신규 코드

xgb_grid = GridSearchCV(estimator = XGBRegressor(), param_grid = hyperparams, 
                verbose=True, scoring=rmsle_scorer, cv=5, n_jobs=-1)

xgb_grid.fit(X_train, y_train)
print(xgb_grid.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
{'colsample_bytree': 0.7, 'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 4, 'n_estimators': 20, 'nthread': 4, 'subsample': 0.7}


In [37]:
xgb_model = xgb_grid.best_estimator_
ml_pred = predict_bikecount(xgb_model, X_train.columns)
df_test['count'] = ml_pred
final_df = df_test[['datetime', 'count']].copy()
final_df.to_csv('submissions_xgboost.csv', header=True, index=False)
!kaggle competitions submit -c bike-sharing-demand -f submissions_xgboost.csv -m "Message"

### Model Evaluation Test: Gradient Boosting Regressor

In [39]:
from sklearn.model_selection import GridSearchCV

# 202402 업데이트: 실행시간이 매우 많이 걸리므로, n_estimators 를 테스트용으로 확연히 줄였습니다. 
# 보다 좋은 성능값을 위해 개인 PC 환경에 맞추어, 해당 값을 늘려보시면 좋을 것 같습니다.
# 기존 코드: n_estimators = [100, 150, 200]
n_estimators = [10, 20, 30] # 신규 코드
max_depth = [5, 7, 9]
min_samples_leaf = [8, 10, 12]
learning_rate = [0.1, 0.15, 0.2]
subsample = [0.6, 0.7, 0.8]

hyperparams = {'n_estimators': n_estimators, 'max_depth': max_depth, 
                    'min_samples_leaf': min_samples_leaf,
                    'learning_rate': learning_rate, 'subsample': subsample
              }

gb_grid=GridSearchCV(estimator = GradientBoostingRegressor(), param_grid = hyperparams, 
                verbose=True, scoring=rmsle_scorer, cv=5, n_jobs=-1)

gb_grid.fit(X_train, y_train)
print(gb_grid.best_params_)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
{'learning_rate': 0.15, 'max_depth': 9, 'min_samples_leaf': 8, 'n_estimators': 30, 'subsample': 0.8}


In [41]:
gb_model = gb_grid.best_estimator_
ml_pred = predict_bikecount(gb_model, X_train.columns)
df_test['count'] = ml_pred
final_df = df_test[['datetime', 'count']].copy()
final_df.to_csv('submissions_gb.csv', header=True, index=False)
!kaggle competitions submit -c bike-sharing-demand -f submissions_gb.csv -m "Message"

<div class="alert alert-block" style="border: 1px solid #455A64;background-color:#ECEFF1;">
본 자료 및 영상 컨텐츠는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 컨텐츠 및 컨텐츠 일부 문구등을 외부에 공개, 게시하는 것을 금지합니다. 특히 자료에 대해서는 저작권법을 엄격하게 적용하겠습니다.
</div>